In [66]:
import pandas as pd
from scipy.interpolate import interp1d
import numpy as np

H_0 = 65 # Hubble constant 

def phillips_relation(delta_m_15):
    M_max_V = -19.504 + 0.736*delta_m_15 + 0.182*(delta_m_15)**2 + 5*np.log10(H_0)
    return M_max_V


def other_phillips_rel(delta_m_15):
    M_max_B = -21.726 + 2.698*delta_m_15
    return M_max_B


# File containing the supernova data
file_path = "Supernova_Ia/table4.dat"

# Step 1: Read the file into a pandas DataFrame
# Adjust `delim_whitespace=True` for space-separated values
columns = ["prefix", "SN", "JD", "U_mag", "U_err", "B_mag", "B_err", "V_mag", "V_err", "R_mag", "R_err", "I_mag", "I_err", "Source"]
df = pd.read_csv(file_path, comment="#", sep="[ \t]+", engine="python", header=0,
                 names=columns)


# filter out unneeded magnitudes as the phillips relation is in B mag (U_mag because if NaN values occur everything gets shifted to the left)
UB_filter = df[["SN", "JD", "U_mag", "U_err", "B_mag", "B_err"]].dropna()


# get rid of the U mag too
B_filter = UB_filter[["SN", "JD", "B_mag", "B_err"]]


unique_sne = B_filter["SN"].unique()


# group the different SNe
sn_group = B_filter.groupby("SN")

for sn in unique_sne:
    sn_data = B_filter[B_filter["SN"] == sn]
    #print(sn_data)

    jd = sn_data["JD"].values
    b_mag = sn_data["B_mag"].values
    #print(b_mag)
    
    interp_func = interp1d(jd, b_mag, kind="linear", fill_value="extrapolate")

    # Find time of maximum brightness and corresponding magnitude
    max_idx = np.argmin(b_mag)  # Index of maximum brightness
    jd_max = jd[max_idx]
    m_b_max = float(b_mag[max_idx])

    # Calculate B-magnitude 15 days after maximum
    m_b_15 = float(interp_func(jd_max + 15))
    #print(m_b_15)

    # Calculate ∆m15
    delta_m15 = m_b_15 - m_b_max
    #print(delta_m15)
        
    

    print("SN", sn)
    print("max B_mag:", m_b_max)
    M_V = phillips_relation(delta_m15)
    M_B = other_phillips_rel(delta_m15) # makes more sense
    print("Multi-color:", M_V)
    print("Phillips:", M_B)

    

    print()

SN 1997E
max B_mag: 15.62
Multi-color: -8.955686184828982
Phillips: -17.741823516507154

SN 1997Y
max B_mag: 15.379
Multi-color: -9.719324669867477
Phillips: -19.52875108030319

SN 1997bp
max B_mag: 14.085
Multi-color: -9.299541112209981
Phillips: -18.50080358125243

SN 1997bq
max B_mag: 14.824
Multi-color: -10.129853577996029
Phillips: -20.689602495548257

SN 1997br
max B_mag: 14.317
Multi-color: -10.396087541407661
Phillips: -21.569354119989796

SN 1997cn
max B_mag: 17.487
Multi-color: -9.337667017382687
Phillips: -18.589085615141677

SN 1997cw
max B_mag: 16.892
Multi-color: -8.960308963098962
Phillips: -17.75162209959883

SN 1997dg
max B_mag: 17.178
Multi-color: -9.686660703688373
Phillips: -19.44386364709665

SN 1997do
max B_mag: 14.626
Multi-color: -10.328684003785694
Phillips: -21.33409683794899

SN 1997dt
max B_mag: 15.402
Multi-color: -9.326743528228784
Phillips: -18.56369940623588

SN 1998D
max B_mag: 18.588
Multi-color: -10.132074094556822
Phillips: -20.69644645551203

SN 199